## PySpark DataFrames: Análisis del Censo

### **Introducción y objetivos del práctico**

Este *notebook* de PySpark vamos a trabajar con información del Censo 2023 de Uruguay.

Enlace de los *Microdatos Censo 2023 Anonimizados*: https://www.gub.uy/instituto-nacional-estadistica/politicas-y-gestion/microdatos-censo-2023-anonimizados


Conjunto que vamos a trabajar en esta clase: https://www5.ine.gub.uy/documents/CENSO%202023/Microdatos/personas_ext_26_02.rar


Cuestionario del censo: https://www.gub.uy/instituto-nacional-estadistica/sites/instituto-nacional-estadistica/files/2025-02/Cuestionario_censo2023%20%281%29.pdf


Diccionario de variables: https://www5.ine.gub.uy/documents/CENSO%202023/Microdatos/Diccionario%20de%20variables%202023.xlsx



Resumen del Proceso

- **Preparar el entorno:** Importar las bibliotecas necesarias y crear la sesión Spark.
- **Leer y preparar los datos:** Leer el archivo CSV, renombrar las columnas y transformar las características en un solo vector.
- **Dividir los datos:** Separar los datos en conjuntos de entrenamiento y prueba.
- **Entrenar el modelo:** Crear y entrenar el modelo de regresión lineal.
- **Evaluar el modelo:** Realizar predicciones y evaluar el rendimiento del modelo.

Con estos pasos, podrás construir y evaluar un modelo de regresión lineal en Spark.

In [ ]:
# !pip install pyspark pandas


### **Configuración del entorno (SparkSession)**


In [ ]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import avg, max, min, round, count, col
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import expr
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, RobustScaler
from pyspark.sql.functions import desc
from pyspark.sql.functions import corr
from pyspark.ml.evaluation import RegressionEvaluator




### **Carga y exploración inicial de datos**


In [ ]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "2g") \
    .appName("MyApp") \
    .getOrCreate()

In [6]:
sdf = spark.read.csv('personas_ext_26_02.csv', header=True, inferSchema=True)

In [ ]:
#!apt-get update -y
#!apt-get install -y openjdk-17-jdk


#!pip install -q "pyspark>=3.6,<3.7"


#import os

#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
#os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

#os.environ.pop("PYSPARK_SUBMIT_ARGS", None)
#os.environ.pop("SPARK_HOME", None)

In [7]:
print("Esquema del DataFrame de Spark:")
sdf.printSchema()

Esquema del DataFrame de Spark:
root
 |-- _c0: integer (nullable = true)
 |-- ID_CENSO: double (nullable = true)
 |-- DIRECCION_ID: string (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- LOCALIDAD: integer (nullable = true)
 |-- VIVID: string (nullable = true)
 |-- HOGID: string (nullable = true)
 |-- PERID: integer (nullable = true)
 |-- REGION_4: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- MUNICIPIO_PAIS: string (nullable = true)
 |-- TIPO_MUNICIPIO_PAIS: string (nullable = true)
 |-- FUENTE_EXT: integer (nullable = true)
 |-- SIT_CALLE: integer (nullable = true)
 |-- CUESTIONARIO_COMPLETO: integer (nullable = true)
 |-- CUESTIONARIO_BASICO: integer (nullable = true)
 |-- RRAA: integer (nullable = true)
 |-- UNIVERSO: integer (nullable = true)
 |-- VIVVO00: integer (nullable = true)
 |-- PERPH02: integer (nullable = true)
 |-- PERNA01: integer (nullable = true)
 |-- PERNA01_TRAMO: string (nullable = true)
 |-- PERPA01: integer (nullable = t

In [8]:
sdf.show(5)

+---+--------+------------+------------+---------+-----+-----+-----+--------+----+--------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+-------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------+
|_c

### **Selección y filtrado de datos**

In [9]:
sdf.select('MUNICIPIO_PAIS').show(20)

+------------------+
|    MUNICIPIO_PAIS|
+------------------+
|       Municipio E|
|       Municipio E|
|       Municipio D|
|       Municipio D|
|       Municipio D|
|     Sin Municipio|
|       Las Piedras|
|       Municipio A|
|       Municipio A|
|     Sin Municipio|
|          Progreso|
|Ciudad de la Costa|
|        Piriápolis|
|     Sin Municipio|
|       Municipio C|
|     Sin Municipio|
|       Municipio E|
|       Municipio E|
|       Municipio E|
|       Municipio F|
+------------------+
only showing top 20 rows



In [10]:
sdf.select('MUNICIPIO_PAIS').distinct().show()

+--------------------+
|      MUNICIPIO_PAIS|
+--------------------+
|            Progreso|
|              Garzón|
|           Rodríguez|
|               Sauce|
|      Joaquín Suarez|
|        San Bautista|
|              Casupá|
|               Pando|
|         Municipio B|
|         Municipio F|
|Nicolich Ciudad L...|
|    Ciudad del Plata|
|   Ombúes de Lavalle|
|         Bella Unión|
|Quebracho - Paysandu|
|          Santa Rosa|
|           Castillos|
|           Mariscala|
|          San Carlos|
|   Florencio Sánchez|
+--------------------+
only showing top 20 rows



In [11]:
sdf.filter(sdf['MUNICIPIO_PAIS'] == "Municipio E").show(20)

+------+--------+------------+------------+---------+-----+-----+-----+--------+----+--------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+---------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------

In [12]:
sdf.filter(sdf['DEPARTAMENTO'] < 4).show(20)

+---+--------+------------+------------+---------+-----+-----+-----+--------+----+------------------+-------------------+----------+---------+---------------------+-------------------+----+--------+-------+-------+-------+-------------+-------+-------+---------+-------+---------+---------+---------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---------+---------+-------+-------+---------+-------+---------+---------+-------+---------+---------+-------+-------+-------+-------+-------+---------+---------+-------+---------+-------+-------+---------+-----------+---------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+---------+---------+-------+---------+---------+-------+---------+---------+-------+---------+-------+--------+--------+------------------------+-----------+----------+---------------+----------+---------------+------------+-----------------+-------+------------+

### **Transformación de columnas**

In [13]:
sdf_renombrado = sdf.select(
    col("ID_CENSO"),
    col("DEPARTAMENTO"),
    col("PERNA01").alias("Edad"),
    col("PERFM01").alias("Sexo"),
    col("MUNICIPIO_PAIS").alias("Municipio")
)

In [14]:
print("Esquema con columnas renombradas:")
sdf_renombrado.printSchema()

Esquema con columnas renombradas:
root
 |-- ID_CENSO: double (nullable = true)
 |-- DEPARTAMENTO: integer (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: integer (nullable = true)
 |-- Municipio: string (nullable = true)



In [15]:
print("Primeras filas con columnas renombradas:")
sdf_renombrado.show(5)


Primeras filas con columnas renombradas:
+--------+------------+----+----+-----------+
|ID_CENSO|DEPARTAMENTO|Edad|Sexo|  Municipio|
+--------+------------+----+----+-----------+
|     1.0|           1|  74|8888|Municipio E|
|     2.0|           1|  37|7777|Municipio E|
|     3.0|           1|  39|8888|Municipio D|
|     4.0|           1|  14|8888|Municipio D|
|     5.0|           1|  19|8888|Municipio D|
+--------+------------+----+----+-----------+
only showing top 5 rows



In [16]:
sdf_con_grupo = sdf_renombrado.withColumn(
    "Grupo_Edad",
    when(col("Edad") < 18, "Menor")
    .when((col("Edad") >= 18) & (col("Edad") < 65), "Adulto")
    .when(col("Edad") >= 65, "Adulto Mayor")
    .otherwise("Sin Dato")
)

In [17]:
print("DataFrame con la nueva columna 'Grupo_Edad':")
sdf_con_grupo.select("Edad", "Grupo_Edad").show(10)

DataFrame con la nueva columna 'Grupo_Edad':
+----+------------+
|Edad|  Grupo_Edad|
+----+------------+
|  74|Adulto Mayor|
|  37|      Adulto|
|  39|      Adulto|
|  14|       Menor|
|  19|      Adulto|
|5555|Adulto Mayor|
|  61|      Adulto|
|  30|      Adulto|
|  52|      Adulto|
|  52|      Adulto|
+----+------------+
only showing top 10 rows



In [18]:
sdf_con_grupo.select("Edad").describe().show()


+-------+------------------+
|summary|              Edad|
+-------+------------------+
|  count|            962034|
|   mean|   47.053906618685|
| stddev|192.84742490705082|
|    min|                 0|
|    max|              5555|
+-------+------------------+



## **Limpieza y Filtrado de Valores Atípicos/Especiales**

Sugerencia:

  - Revisar outliers (valores atípicos).

  - Revisar valores perdidos (missing data).

Estrategias de manejo de valores perdidos:
- imputación por el promedio (distribución normal), o por la mediana (distribución paranormal)

- imputación por distancia de mahalanobis: https://www.delftstack.com/es/howto/python/python-mahalanobis-distance/

- eliminar (tener cuidado cuando tienen pocos datos)

Criterio para imputar valores:
- Regla general (20%): Si una persona tiene más del 20% de sus respuestas perdidas en una escala o variable, sus datos deben ser considerados no fiables (posible recomendación eliminar ese caso según: Enders, 2022; Hughes et al., 2019;
Jakobsen et al., 2017)


*Referencias bibliograficas*

- Enders, C. K. (2022). Applied missing data analysis. Guilford Publications

- Hughes, R. A., Heron, J., Sterne, J. A. C., & Tilling, K. (2019). Accounting for missing data in
statistical analyses: multiple imputation is not always the answer. International Journal of
Epidemiology, 48(4), 1294-1304. https://doi.org/10.1093/ije/dyz032

- Jakobsen, J. C., Gluud, C., Wetterslev, J., & Winkel, P. (2017). When and how should multiple
imputation be used for handling missing data in randomised clinical trials – a practical guide
with flowcharts. BMC Medical Research Methodology, 17(1), 162.
https://doi.org/10.1186/s12874-017-0442-1

In [19]:
sdf_limpio = sdf_renombrado.filter(
    (col("Edad") < 120) & (col("Edad") > 0)
)

In [20]:
total_original = sdf.count()
total_limpio = sdf_limpio.count()

print(f"Total de registros original: {total_original}")
print(f"Total de registros después de filtrar edades atípicas: {total_limpio}")
print(f"Registros eliminados: {total_original - total_limpio}")



Total de registros original: 962034
Total de registros después de filtrar edades atípicas: 953607
Registros eliminados: 8427


### **Análisis descriptivo y agregaciones**

Cálculo de la edad promedio, máxima y mínima por municipio

In [23]:
stats_por_municipio = sdf_limpio.groupBy("Municipio").agg(
    round(avg("Edad"), 2).alias("Edad_Promedio"),
    max("Edad").alias("Edad_Maxima"),
    min("Edad").alias("Edad_Minima"),
    count("*").alias("Total_Personas")
)

stats_por_municipio.orderBy(col("Edad_Promedio").desc()).show(10)

+-----------------+-------------+-----------+-----------+--------------+
|        Municipio|Edad_Promedio|Edad_Maxima|Edad_Minima|Total_Personas|
+-----------------+-------------+-----------+-----------+--------------+
|             Soca|         72.4|         83|         63|             5|
| La Paz - Colonia|         70.0|         70|         70|             1|
|         Porvenir|         70.0|         73|         67|             2|
|            Pando|         66.0|         88|         25|            13|
|    Empalme Olmos|        64.14|         85|         34|             7|
|Florencio Sánchez|         64.0|         66|         62|             2|
|        Rodríguez|         64.0|         78|         43|             3|
|            Young|         61.5|         91|         20|             8|
|       Río Branco|         60.5|         86|         26|            12|
|             Tala|        60.17|         82|         29|             6|
+-----------------+-------------+-----------+------

PySpark SQL: Unificación de SQL con Spark

In [24]:
sdf_renombrado.createOrReplaceTempView("censo_personas")

print("Ejecutando consulta SQL: Conteo de personas por Municipio:")

consulta_sql = """
SELECT
    Municipio,
    COUNT(*) AS Total_Personas
FROM
    censo_personas
GROUP BY
    Municipio
ORDER BY
    Total_Personas DESC
"""

resultado_sql = spark.sql(consulta_sql).show()


Ejecutando consulta SQL: Conteo de personas por Municipio:
+--------------------+--------------+
|           Municipio|Total_Personas|
+--------------------+--------------+
|         Municipio A|        157161|
|         Municipio D|        130397|
|         Municipio B|        130220|
|         Municipio E|        128266|
|         Municipio C|        126297|
|         Municipio G|        119946|
|        Municipio CH|         93272|
|         Municipio F|         72914|
|       Sin Municipio|          1114|
|                9898|           372|
|  Ciudad de la Costa|           215|
|      Punta del Este|           174|
|           Maldonado|           165|
|         La Floresta|           150|
|          Piriápolis|           149|
|             Salinas|            97|
|Parque del Plata ...|            70|
|    Ciudad del Plata|            52|
|          San Carlos|            50|
|         Las Piedras|            41|
+--------------------+--------------+
only showing top 20 rows



### **Regresión**

¿Existe relación entre la edad de la persona (PERNA01) y su nivel educativo alcanzado (NIVELEDU)?

PERNA01: edad (numérica continua).

NIVELEDU: nivel educativo (ordinal, pero codificado numéricamente).



![Modelos](https://th.bing.com/th/id/R.b6b5a3a73f568bd1f2b588feb93cd02a?rik=d0gt%2bmbw7VaQOw&riu=http%3a%2f%2fwww.favouriteblog.com%2fwp-content%2fuploads%2f2017%2f07%2fTypes-of-Learning.png&ehk=z%2btK2BpOxRNtB3u4Gis9bn3dIkKxuVwcqmENF9cGlx0%3d&risl=&pid=ImgRaw&r=0)


En esta práctica, continuaremos explorando los modelos supervisados, aquellos en los que el modelo aprende a partir de datos etiquetados. Estos modelos se dividen en:

- Regresión Lineal: Se usa cuando la variable objetivo es continua. Ejemplo: Predecir el precio de una vivienda en función de su superficie y ubicación.

-  Regresión Logística: Aunque su nombre sugiere "regresión", este modelo se usa para clasificación binaria o multiclase.
    - Calcula la probabilidad de que una observación pertenezca a una clase determinada.
    - Se usa en problemas como detección de spam, diagnóstico médico y predicción de abandono de clientes.

- Support Vector Machine (SVM): Es un modelo de clasificación que encuentra el hiperplano óptimo que separa las clases.
    - Funciona bien en problemas con datos complejos y espacios de alta dimensión.
    - Se usa en problemas como reconocimiento de imágenes y detección de fraudes.

Vamos a investigar las caracteristicas de la variable **NIVELEDU**


La siguiente tabla detalla la codificación para el máximo nivel educativo alcanzado (NIVELEDU) según la fuente de datos.

| Código (NIVELEDU) | Descripción (Máximo Nivel Educativo Alcanzado) |
| :---------------: | :--------------------------------------------- |
| 0                 | Menor de 4 años                                |
| 1                 | Preescolar                                     |
| 2                 | Primaria común o especial                      |
| 4                 | Educación media básica o Ciclo Básico          |
| 5                 | Educación media superior o Bachillerato        |
| 6                 | Capacitaciones o cursos de UTU                 |
| 7                 | Magisterio o profesorado                       |
| 8                 | Terciario no universitario                     |
| 9                 | Universidad o similar                          |
| 10                | Postgrado (Diploma/Maestría/Doctorado)         |
| 12                | Nunca asistió                                  |

In [31]:
sdf_reg = sdf.select(
    col("NIVELEDU"),
    col("PERNA01")
    )

Ver el tipo con schema detallado

In [32]:
sdf_reg.printSchema()

root
 |-- NIVELEDU: integer (nullable = true)
 |-- PERNA01: integer (nullable = true)



Revisar estadísticos descriptivos

In [33]:
sdf_reg.describe().show()

+-------+------------------+------------------+
|summary|          NIVELEDU|           PERNA01|
+-------+------------------+------------------+
|  count|            962034|            962034|
|   mean|108.26894163823732|   47.053906618685|
| stddev|1002.6223512644426|192.84742490705082|
|    min|                 0|                 0|
|    max|              9898|              5555|
+-------+------------------+------------------+



Contar cuántos valores distintos existen en la columna

In [34]:
sdf_reg.select(countDistinct('NIVELEDU')).show()

+------------------------+
|count(DISTINCT NIVELEDU)|
+------------------------+
|                      13|
+------------------------+



Contar frecuencia de cada valor

In [35]:
sdf_reg.groupBy('NIVELEDU').count().orderBy('NIVELEDU').show()


+--------+------+
|NIVELEDU| count|
+--------+------+
|       0| 30754|
|       1| 19056|
|       2|193513|
|       4|205751|
|       5|184029|
|       6| 13459|
|       7| 24630|
|       8| 38446|
|       9|188730|
|      10| 35895|
|      12| 17713|
|    8888|   372|
|    9898|  9686|
+--------+------+



Excluir los valores
|      12|
|    8888|
|    9898|

In [36]:
sdf_reg = sdf_reg.filter(~col("NIVELEDU").isin([12, 8888, 9898]))


Verificar

In [37]:
sdf_reg.select('NIVELEDU').distinct().orderBy('NIVELEDU').show()


+--------+
|NIVELEDU|
+--------+
|       0|
|       1|
|       2|
|       4|
|       5|
|       6|
|       7|
|       8|
|       9|
|      10|
+--------+



Vamos a investigar las caracteristicas de la variable **PERNA01**


Variable que tiene información de la edad en años.

Descripción:
- Edad en años cumplidos. Las personas residentes en localidades con menos de 10.000 habitantes no tienen dato en esta variable por secreto estadístico.

- Población estimada (excluyendo los residentes en localidades de menos de 10.000 habitantes)


In [28]:
sdf_reg.describe().show()

+-------+------------------+------------------+
|summary|          NIVELEDU|           PERNA01|
+-------+------------------+------------------+
|  count|            962034|            962034|
|   mean|108.26894163823732|   47.053906618685|
| stddev|1002.6223512644426|192.84742490705082|
|    min|                 0|                 0|
|    max|              9898|              5555|
+-------+------------------+------------------+



Contar cuántos valores distintos existen en la columna

In [38]:
sdf_reg.select(countDistinct('PERNA01')).show()

+-----------------------+
|count(DISTINCT PERNA01)|
+-----------------------+
|                    109|
+-----------------------+



Ver los valores únicos

In [39]:
sdf_reg.select('PERNA01').distinct().orderBy('PERNA01').show()


+-------+
|PERNA01|
+-------+
|      0|
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
+-------+
only showing top 20 rows



Contar frecuencia de cada valor

In [40]:
sdf_reg.groupBy('PERNA01').count().orderBy('PERNA01').show()

+-------+-----+
|PERNA01|count|
+-------+-----+
|      0| 7266|
|      1| 7571|
|      2| 7768|
|      3| 8133|
|      4| 8326|
|      5| 9158|
|      6|10278|
|      7|10947|
|      8|11249|
|      9|11292|
|     10|11476|
|     11|11404|
|     12|11266|
|     13|11448|
|     14|11338|
|     15|11381|
|     16|11851|
|     17|11470|
|     18|12913|
|     19|13196|
+-------+-----+
only showing top 20 rows



Últimos 30 valores (mayores)

In [42]:
sdf_reg.groupBy('PERNA01').count().orderBy(desc('PERNA01')).show(30)

+-------+-----+
|PERNA01|count|
+-------+-----+
|   5555| 1126|
|    107|    2|
|    106|    1|
|    105|   10|
|    104|    8|
|    103|   16|
|    102|   19|
|    101|   49|
|    100|   75|
|     99|  102|
|     98|  184|
|     97|  239|
|     96|  368|
|     95|  506|
|     94|  692|
|     93|  894|
|     92| 1210|
|     91| 1467|
|     90| 1759|
|     89| 1980|
|     88| 2291|
|     87| 2627|
|     86| 2899|
|     85| 3183|
|     84| 3758|
|     83| 4145|
|     82| 4495|
|     81| 4549|
|     80| 5061|
|     79| 5539|
+-------+-----+
only showing top 30 rows



Excluir los valores menores a 109


In [44]:
sdf_reg = sdf_reg.filter(col("PERNA01") < 109)


Verificamos

In [45]:
sdf_reg.groupBy('PERNA01').count().orderBy(desc('PERNA01')).show(30)

+-------+-----+
|PERNA01|count|
+-------+-----+
|    107|    2|
|    106|    1|
|    105|   10|
|    104|    8|
|    103|   16|
|    102|   19|
|    101|   49|
|    100|   75|
|     99|  102|
|     98|  184|
|     97|  239|
|     96|  368|
|     95|  506|
|     94|  692|
|     93|  894|
|     92| 1210|
|     91| 1467|
|     90| 1759|
|     89| 1980|
|     88| 2291|
|     87| 2627|
|     86| 2899|
|     85| 3183|
|     84| 3758|
|     83| 4145|
|     82| 4495|
|     81| 4549|
|     80| 5061|
|     79| 5539|
|     78| 5953|
+-------+-----+
only showing top 30 rows



Ver coeficientes de correlación

In [46]:
from scipy.stats import pearsonr

sdf_reg.select(corr("NIVELEDU", "PERNA01")).show()

+-----------------------+
|corr(NIVELEDU, PERNA01)|
+-----------------------+
|    0.17654751125739415|
+-----------------------+



In [47]:
pdf = sdf_reg.select("NIVELEDU", "PERNA01").toPandas()

r, p = pearsonr(pdf["NIVELEDU"], pdf["PERNA01"])
print(f"r = {r:.4f}, p = {p:.6f}")

r = 0.1765, p = 0.000000


**Interpretación del coeficiente de correlación de Pearson**

El análisis de correlación de **Pearson** permite cuantificar la **fuerza y dirección** de la relación lineal entre dos variables numéricas.  
En este caso, estamos evaluando la relación entre:

- **NIVELEDU** → nivel educativo (variable dependiente)  
- **PERNA01** → indicador predictor (por ejemplo, una variable socioeconómica o demográfica del censo)

----

**r =** Es el coeficiente de correlación de Pearson. Toma valores entre -1 y +1.

**p =** Es el valor de significancia asociado al test estadístico. Permite evaluar si la correlación observada podría deberse al azar.


Cómo interpretar los resultados:
- Si r > 0, la relación es positiva: cuando aumenta PERNA01, también tiende a aumentar NIVELEDU.
- Si r < 0, la relación es negativa: cuando aumenta PERNA01, NIVELEDU tiende a disminuir.
- Cuanto más cerca de ±1, más fuerte es la relación lineal.
- Un valor cercano a 0 indica ausencia de relación lineal (aunque puede haber relación no lineal).

---
¿Cómo interpretar estos resultados?

Existe una correlación positiva débil pero significativa entre PERNA01 y NIVELEDU.

Esto sugiere que, en promedio, a medida que aumenta PERNA01, el nivel educativo también tiende a aumentar ligeramente, aunque la relación no es fuerte.

### **AHORA SI: Regresión**

Objetivo

- Evaluar si la edad predice el nivel educativo.

En este ejemplo haremos:
- Variable dependiente (Y): NIVELEDU
- Variable independiente (X): PERNA01 (Edad)

### Preparar los Datos para la Regresión Lineal (Sin Estandarización)

En este paso, prepararemos los datos para la regresión lineal utilizando VectorAssembler.


In [48]:
sdf_reg = sdf_reg.withColumn("PERNA01", col("PERNA01").cast("double"))

sdf_reg = sdf_reg.withColumn("NIVELEDU", col("NIVELEDU").cast("double"))

In [49]:
sdf_reg = sdf_reg.na.drop(subset=["PERNA01", "NIVELEDU"])

In [50]:
assembler = VectorAssembler(inputCols=["PERNA01"], outputCol="features")

In [51]:
assembled_no_scale = assembler.transform(sdf_reg)


**Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)**


Para evaluar el rendimiento del modelo, se dividirá el dataset en dos subconjuntos:

🔹 Conjunto de entrenamiento (Train): Usado para entrenar el modelo.

🔹 Conjunto de prueba (Test): Usado para evaluar el rendimiento del modelo.

In [52]:
train_A, test_A = assembled_no_scale.randomSplit([0.8, 0.2], seed=42)

Mostrar la cantidad de registros en cada conjunto

In [53]:
print(f"Training Data Count: {train_A.count()}")

Training Data Count: 746405


In [54]:
print(f"Test Data Count: {test_A.count()}")

Test Data Count: 186732


### Entrenar el Modelo de Regresión Lineal

Ahora, crearemos y entrenaremos el modelo de regresión lineal.

Crear el modelo de regresión lineal

In [55]:
lr_A = LinearRegression(featuresCol="features", labelCol="NIVELEDU")


Entrenar el modelo con los datos de entrenamiento

In [56]:
lr_model_A = lr_A.fit(train_A)

### Evaluar el Modelo

Utilizaremos el conjunto de prueba para evaluar el modelo calculando el Error Cuadrático Medio (RMSE) y el coeficiente de determinación (R2).

In [57]:
pred_A = lr_model_A.transform(test_A)
evaluator = RegressionEvaluator(labelCol="NIVELEDU", predictionCol="prediction")

rmse_A = evaluator.setMetricName("rmse").evaluate(pred_A)
r2_A   = evaluator.setMetricName("r2").evaluate(pred_A)

**Métricas del modelo**

- **RMSE (Root Mean Squared Error):** mide el error promedio de las predicciones del modelo. Valores más bajos indican un mejor ajuste.  

- **R² (Coeficiente de determinación):** representa la proporción de la variabilidad de **NIVELEDU** que puede explicarse a partir de **PERNA01**. Un R² más cercano a 1 indica un mejor ajuste del modelo.

### Inspeccionar los Coeficientes del Modelo

Examinar los coeficientes e intercepto del modelo

In [58]:
print(f"RMSE (test): {rmse_A:.6f}")
print(f"R²   (test): {r2_A:.6f}")
print(f"Intercepto: {lr_model_A.intercept:.6f}")
print("Coeficientes:", lr_model_A.coefficients)

RMSE (test): 2.764512
R²   (test): 0.030593
Intercepto: 4.234646
Coeficientes: [0.021760639212818806]


**Resultados del modelo de regresión lineal**

*Coeficientes del modelo*

| Variable | Coeficiente | Interpretación |
|-----------|-------------|----------------|
| PERNA01 | 0.021 | Por cada unidad que aumenta el valor de **PERNA01**, el nivel educativo promedio (**NIVELEDU**) aumenta en aproximadamente **0.021 puntos**, manteniendo constante el resto de variables. Esto indica una **relación positiva**: a mayor valor en PERNA01, ligeramente mayor nivel educativo. |



*Intercepto del modelo*

| Intercepto | Valor | Interpretación |
|-------------|--------|----------------|
| Intercepto | 4.32 | Cuando **PERNA01 = 0**, el valor estimado del nivel educativo (**NIVELEDU**) es **4.23**. Este representa el valor base del modelo, es decir, el punto donde la recta de regresión corta el eje Y. |

Interpretación general

El modelo muestra una **relación lineal positiva y débil** entre **PERNA01** y **NIVELEDU**: a medida que **PERNA01** aumenta, el nivel educativo promedio también tiende a incrementarse ligeramente.  

El **R²** obtenido permite estimar cuánta variabilidad del nivel educativo se explica por **PERNA01**.

Si el valor de R² es bajo, significa que aunque existe una tendencia positiva, **la variable PERNA01 por sí sola no explica una gran proporción de las diferencias en el nivel educativo**.

Si el rendimiento del modelo no cumple con sus expectativas, puede probar las siguientes estrategias para mejorarlo:

- **Escalado de funciones:** estandarice o normalice las funciones de entrada para garantizar que estén en la misma escala.

- **Ajuste de hiperparámetros:** ajuste los hiperparámetros del modelo, como la intensidad de la regularización o el recuento de iteraciones.

---
---

---

---

## **Parte  II: MODELO B: Regresión con ROBUST SCALER**

Generar vector crudo para escalar


In [59]:
assembler_raw = VectorAssembler(inputCols=["PERNA01"], outputCol="raw_features")
assembled_raw = assembler_raw.transform(sdf_reg)

Split reproducible antes de escalar

In [60]:
train_raw, test_raw = assembled_raw.randomSplit([0.8, 0.2], seed=42)

In [61]:
print(f"Training Data Count: {train_raw.count()}")
print(f"Test Data Count: {test_raw.count()}")

Training Data Count: 746405
Test Data Count: 186732


Ajustar RobustScaler SOLO con train y transformar ambos

In [62]:
scaler = RobustScaler(inputCol="raw_features", outputCol="features")
scaler_model = scaler.fit(train_raw)

train_B = scaler_model.transform(train_raw)
test_B  = scaler_model.transform(test_raw)

Regresión lineal sobre features escaladas

In [63]:
lr_B = LinearRegression(featuresCol="features", labelCol="NIVELEDU")
lr_model_B = lr_B.fit(train_B)

pred_B = lr_model_B.transform(test_B)

rmse_B = evaluator.setMetricName("rmse").evaluate(pred_B)
r2_B   = evaluator.setMetricName("r2").evaluate(pred_B)

Resultados MODELO B (RobustScaler)

In [64]:
print(f"RMSE (test, escalado): {rmse_B:.6f}")
print(f"R²   (test, escalado): {r2_B:.6f}")
print(f"Intercepto: {lr_model_B.intercept:.6f}")
print("Coeficientes:", lr_model_B.coefficients)

RMSE (test, escalado): 2.764512
R²   (test, escalado): 0.030593
Intercepto: 4.234646
Coeficientes: [0.8051436508804143]


Comparativa rápida

In [65]:
print(f"A) Sin estandarizar  -> RMSE: {rmse_A:.6f} | R²: {r2_A:.6f}")
print(f"B) RobustScaler      -> RMSE: {rmse_B:.6f} | R²: {r2_B:.6f}")

A) Sin estandarizar  -> RMSE: 2.764512 | R²: 0.030593
B) RobustScaler      -> RMSE: 2.764512 | R²: 0.030593


# **Interpretación**

Probamos dos versiones del mismo modelo de regresión lineal:

  A) Sin estandarizar los datos.

  B) Con los datos escalados mediante RobustScaler, una técnica que reduce la influencia de valores atípicos.

El objetivo es ver si la estandarización mejora el rendimiento del modelo.

----

**RMSE (Error cuadrático medio):** Mide cuánto se alejan las predicciones de los valores reales.

- En ambos casos, el error es idéntico (2.76) → no hubo mejora.

**R² (Coeficiente de determinación):** Indica qué proporción de la variabilidad de la variable dependiente explica el modelo.

- Un valor de 0.03 significa que el modelo explica solo el 3% de la variación.

Esto sugiere que el modelo tiene baja capacidad predictiva, posiblemente porque las variables explicativas no son buenas predictores o la relación no es lineal.

---
En este análisis, los resultados del modelo con y sin estandarización (RobustScaler) fueron idénticos tanto en el RMSE como en el R².

Esto indica que la escala de las variables no afectó el desempeño del modelo.


In [66]:
spark.stop()

Referencias para profundizar en este campo:


[Pyspark Tutorial: Getting Started with Pyspark](https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark)

[Linear Regression in PySpark](https://medium.com/@roshmitadey/a-comprehensive-guide-to-linear-regression-in-pyspark-810fdaf5c17c)

[Regresión lineal de PySpark: cómo crear y evaluar modelos de regresión lineal utilizando PySpark MLlib](https://www.machinelearningplus.com/pyspark/pyspark-linear-regression/?utm_content=cmp-true)










Regresión lineal: https://cienciadedatos.net/documentos/py10-regresion-lineal-python.html

Outliers: https://www.delftstack.com/es/howto/python/outlier-detection-python/
